## Recursive inference (multi-step) for BSCTRFM models

## this script:
### uses time series instead of SLDB arrays for easier and more efficient timestamp management
### uses raw data (no outlier removal)

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [3]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        sequential_id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    sequential_id_flat_array = sequential_id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'sequential_id': _float_feature_from_list_of_values(sequential_id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [4]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [5]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [6]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [7]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 168

In [8]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [9]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [10]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [11]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'PEMS-SF_SEPARATED_FULL'

In [12]:
m = 168
m

168

In [13]:
t = 168
t

168

In [14]:
model_ids = [
    'BSCTRFM_TPU_079',
    'BSCTRFM_TPU_079',
    'BSCTRFM_TPU_079',
    'BSCTRFM_TPU_079',
    'BSCTRFM_TPU_079',
]

In [15]:
executions = [
    5,
    6,
    7,
    8,
    9,
]

In [16]:
saved_model_ids = [
    '1649435365',
    '1649435721',
    '1649436072',
    '1649436430',
    '1649436784',
]

In [17]:
for model_id, execution, saved_model_id in zip(model_ids,
                                               executions,
                                               saved_model_ids):
    print(model_id, execution, saved_model_id)

BSCTRFM_TPU_079 5 1649435365
BSCTRFM_TPU_079 6 1649435721
BSCTRFM_TPU_079 7 1649436072
BSCTRFM_TPU_079 8 1649436430
BSCTRFM_TPU_079 9 1649436784


In [18]:
encoder_input_columns = [
    'occupancy_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
]

In [19]:
decoder_input_columns = encoder_input_columns

In [20]:
id_columns = ['sequential_id']

In [21]:
# get and test the starting indexes list to produce 7 hourly-day-ahead prediction sets for benchmarking

# dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]

In [22]:
data_folder = '{}/datasets/traffic/PEMS-SF'.format(PROJECT_ROOT)
data_folder

'/home/developer/gcp/cbidmltsf/datasets/traffic/PEMS-SF'

In [23]:
# get the station ids list from the dataset folder

In [24]:
def process_list(s, variable_type=int, delimiter=None):
    """Parses a line in the PEMS format to a list."""
    if delimiter is None:
      l = [
          variable_type(i) for i in s.replace('[', '').replace(']', '').split()
      ]
    else:
      l = [
          variable_type(i)
          for i in s.replace('[', '').replace(']', '').split(delimiter)
      ]

    return l

In [25]:
def read_single_list(filename):
    """Returns single list from a file in the PEMS-custom format."""
    with open(os.path.join(data_folder, filename), 'r') as dat:
        l = process_list(dat.readlines()[0])
    return l

In [26]:
station_ids = [id for id in read_single_list('stations_list')]
len(station_ids)

963

In [27]:
# test time series and scalers are loaded just once, and kept in memory to speed up inference

ts_test = dict()
scaler = dict()

for station_id in station_ids:

    # read the time series for the current sensor station
    ts_test[station_id] = pd.read_pickle('{}/test/ST_{}.pkl'.format(data_dir, station_id))

    # report start and end timestamp for the loaded time series
    # print('Loaded test time series for {}, which spans from index {} to index {}'.\
    #      format(station_id,
    #             str(ts_test[station_id].index[:1][0]),
    #             str(ts_test[station_id].index[-1:][0])))

    # build a path to the scaler of the time series
    scaler_path = '{}/scalers/min_max_{}.save'.format(data_dir, station_id)
    # and load it
    scaler[station_id] = joblib.load(scaler_path)


In [28]:
len(ts_test.keys()), len(scaler.keys())

(963, 963)

## skip the following code to avoid inference process

In [29]:
# a second inference identifier to run more than one inference
# on the same combination model_id, execution, inference;
# to produce different inference processes because predict_fn is stochastic
for event in [0]:

    for model_id, execution, saved_model_id in zip(model_ids,
                                                   executions,
                                                   saved_model_ids):
        # print(saved_model_id, model_id, execution)

        # use model identifier and execution number to build the model directory string
        model_dir = '{}_{:02d}'.format(model_id, execution)

        # get the path to the saved model main directory
        saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                             'models',
                                                             model_dir)

        # build the full path for the latest saved model dir
        export_dir = '{}/{}'.format(saved_model_path, saved_model_id)
        print ('Exported model path is {}'.format(export_dir))

        # load the saved model and the prediction function
        imported = load(export_dir=export_dir, tags='serve')
        predict_fn = imported.signatures["serving_default"]


        # use a dictionary to remain the code consistent with the SLDB building process
        # most of the times, only ts['test'] will be used for inference
        # however, ts['eval'] might also be used, as it have not really been seen by training process
        # (no modification in model training resulted from evaluation stage)

        # a dictionary to store predictions detail dataframe per station id
        # predictions_detail = dict()

        # a columns list for the predictions dataframe
        pred_df_columns = [
            'model_id',
            'execution',
            'saved_model_id',
            'dataset',
            'inference',
            'event',
            'station_id',
            # no date information in the traffic dataset, then
            'ts_index',
            'prediction',
            'target'
        ]

        # a dataframe for global inferences
        global_inferences_df = pd.DataFrame(columns=pred_df_columns)

        for station_id in station_ids:

            # build the predictions dataframe as a key-value pair of the dictionary
            # predictions_detail[station_id] = pd.DataFrame(columns=pred_df_columns)

            # iterate on a set of valid rows of the test dataset
            # starting_point = 0 # based on the inference dataset
            # span = 1 + 6*24 # number of days in the test dataset, expressed in hours

            # MAKE THE PREDICTION SETS FOR BENCHMARKING ONLY! 
            # dataset_row_indexes_list = starting_point + np.arange(span)
            
            # uncomment the following line to restart predictions at each day start
            # dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]
            
            # uncomment the following line to restart predictions at each week start
            dataset_row_indexes_list = [0]
           

            for start_index in dataset_row_indexes_list:

                # define first prediction interval with start- and end-index
                # given the interval time_series[start_index:end_index]
                # the conditioning range is the union of the encoder-input and the decoder-input
                # and the prediction range is only the last lecture in the interval,
                # by means of a recursive inference process
                # on each step the last prediction is added to the decoder input
                # and the prediction range grows one step into the future

                # get the end-index of this recursive inference interval
                end_index = start_index + (m + t)

                # initialize a list to store recurrent predictions for this interval
                predictions_list = list()

                for i in np.arange(forecast_window):

                    # build the inference interval as a sub-series of the dataset
                    # the following slicing operation is equivalent to direct slicing as in:
                    # sub_series = ts_test[station_id][start_index + i : end_index + i]
                    sub_series = ts_test[station_id].iloc[start_index + i : end_index + i]

                    # important: build sources as copies of the sub-series (and therefore of the global time series)
                    # to avoid overwriting the original dataset

                    # the encoder input source
                    encoder_input = sub_series[encoder_input_columns][:m].copy()

                    # the decoder input source
                    # decoder_input = sub_series[m-1:-1].copy()
                    decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()

                    # the id (integer) for the sensor station
                    id_input = sub_series[id_columns][:1].copy()

                    # on first step (i=0), the decoder input carries only true values
                    # and the predictions list is empty
                    # on subsequent steps, the decoder input includes all previous predictions
                    # (stored in the predictions list)
                    if i > 0:
                        decoder_input['occupancy_scaled'][-i:] = predictions_list

                    # the target source, for metrics calculation
                    # the first part of the sub-series is the encoder input, and
                    # the second part of the sub-series is the target (only the variable column!)
                    target = sub_series['occupancy_scaled'][m:].copy()

                    # build source tensors from the sub-series    
                    encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
                    decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
                    id_tensor = tf.expand_dims(id_input, axis=0)

                    # make input example for the prediction function
                    input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                        decoder_input_tensor,
                                                                        id_tensor)

                    # get the output of the prediction function as a dictionary
                    predict_output_dict = predict_fn(input_example)

                    # get the prediction output tensor
                    predict_output_tensor = predict_output_dict['forecast']

                    # get the most recent prediction
                    most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

                    # append the most recent prediction timestep to the predictions list
                    predictions_list.append(most_recent_prediction)

                # iterative predictions over the forecast window reside in predictions_list
                # convert list to array, then expand feature dimension with value 1
                predicted_values = np.array(predictions_list).reshape(-1, 1)

                # inverse-scale predictions
                rescaled_predicted_values = scaler[station_id].inverse_transform(predicted_values)

                # and the true values remain in the prediction tensor, pass them to a NumPy array
                # for the true values array, expand feature dimension with value 1
                true_values = np.array(target[-i-1:]).reshape(-1, 1)

                # inverse-scale true values
                rescaled_true_values = scaler[station_id].inverse_transform(true_values)

                # a temporary dataframe built from the data in the current row
                df = pd.DataFrame(columns=pred_df_columns)
                df['model_id'] = forecast_window*[model_id]
                df['execution'] = forecast_window*[execution]
                df['saved_model_id'] = forecast_window*[saved_model_id]
                df['dataset'] = forecast_window*[dataset]
                df['inference'] = forecast_window*[inference]
                df['event'] = forecast_window*[event]
                df['station_id'] = forecast_window*[station_id]
                df['ts_index'] = target.index[-i-1:]
                df['prediction'] = np.squeeze(rescaled_predicted_values)
                df['target'] = np.squeeze(rescaled_true_values)

                # append the temporary dataframe to the predictions detail dataframe
                # predictions_detail[station_id] = pd.concat([predictions_detail[station_id], df])
                global_inferences_df = pd.concat([global_inferences_df, df])

            print('Finished inferences for {}'.format(station_id))

            # reset the index of final dataframe, once all of its rows (dataset) have been processed
            # predictions_detail[station_id] = predictions_detail[station_id].reset_index(drop=True)


            # do not persist a single pickle file per customer_id
            persist_separated_pickles = False

            if persist_separated_pickles:

                # build a path to persist the dataframe to database/predictions_detail/
                detail_pickle_path = '{}/{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}_{}.pkl'.format(
                    PROJECT_ROOT,
                    'database',
                    'inferences',
                    'traffic',
                    model_id,
                    execution,
                    saved_model_id,
                    dataset,
                    inference,
                    event,
                    station_id)

                # persist the Pandas dataframe to database/predictions_detail/
                predictions_detail[station_id].to_pickle(detail_pickle_path)
                print(
                    'Persisted predictions detail of {}_{:02d}_{}_{}_{}_{:02d}_{}'.format(model_id,
                                                                                       execution,
                                                                                       saved_model_id,
                                                                                       dataset,
                                                                                       inference,
                                                                                       event,
                                                                                       station_id)
                )
                
        # reset index for global inferences dataframe
        global_inferences_df = global_inferences_df.reset_index(drop=True)

        # build a path to persist the global inferences dataframe
        global_inferences_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
            PROJECT_ROOT,
            'inferences',
            'traffic',
            model_id,
            execution,
            saved_model_id,
            dataset,
            inference,
            event
        )

        # persist the global inference dataframe
        global_inferences_df.to_pickle(global_inferences_pickle_path)
        print('Persisted global inferences to {}'.format(global_inferences_pickle_path))


Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_079_05/export/exporter/1649435365
Finished inferences for 400000
Finished inferences for 400001
Finished inferences for 400009
Finished inferences for 400010
Finished inferences for 400015
Finished inferences for 400017
Finished inferences for 400025
Finished inferences for 400026
Finished inferences for 400027
Finished inferences for 400030
Finished inferences for 400031
Finished inferences for 400035
Finished inferences for 400037
Finished inferences for 400039
Finished inferences for 400040
Finished inferences for 400041
Finished inferences for 400043
Finished inferences for 400044
Finished inferences for 400045
Finished inferences for 400049
Finished inferences for 400052
Finished inferences for 400053
Finished inferences for 400057
Finished inferences for 400059
Finished inferences for 400060
Finished inferences for 400065
Finished inferences for 400067
Finished inferences for 400071
Finished inferences for 40

Finished inferences for 400529
Finished inferences for 400531
Finished inferences for 400534
Finished inferences for 400536
Finished inferences for 400538
Finished inferences for 400539
Finished inferences for 400540
Finished inferences for 400545
Finished inferences for 400546
Finished inferences for 400547
Finished inferences for 400549
Finished inferences for 400551
Finished inferences for 400552
Finished inferences for 400553
Finished inferences for 400554
Finished inferences for 400560
Finished inferences for 400563
Finished inferences for 400565
Finished inferences for 400566
Finished inferences for 400568
Finished inferences for 400570
Finished inferences for 400571
Finished inferences for 400573
Finished inferences for 400574
Finished inferences for 400576
Finished inferences for 400577
Finished inferences for 400578
Finished inferences for 400579
Finished inferences for 400580
Finished inferences for 400581
Finished inferences for 400582
Finished inferences for 400586
Finished

Finished inferences for 401102
Finished inferences for 401103
Finished inferences for 401104
Finished inferences for 401106
Finished inferences for 401107
Finished inferences for 401110
Finished inferences for 401111
Finished inferences for 401112
Finished inferences for 401113
Finished inferences for 401114
Finished inferences for 401118
Finished inferences for 401121
Finished inferences for 401124
Finished inferences for 401126
Finished inferences for 401129
Finished inferences for 401134
Finished inferences for 401137
Finished inferences for 401141
Finished inferences for 401142
Finished inferences for 401143
Finished inferences for 401144
Finished inferences for 401148
Finished inferences for 401151
Finished inferences for 401152
Finished inferences for 401154
Finished inferences for 401155
Finished inferences for 401156
Finished inferences for 401162
Finished inferences for 401163
Finished inferences for 401165
Finished inferences for 401167
Finished inferences for 401176
Finished

Finished inferences for 401683
Finished inferences for 401684
Finished inferences for 401686
Finished inferences for 401688
Finished inferences for 401689
Finished inferences for 401691
Finished inferences for 401692
Finished inferences for 401693
Finished inferences for 401694
Finished inferences for 401695
Finished inferences for 401696
Finished inferences for 401697
Finished inferences for 401698
Finished inferences for 401699
Finished inferences for 401700
Finished inferences for 401701
Finished inferences for 401702
Finished inferences for 401703
Finished inferences for 401704
Finished inferences for 401705
Finished inferences for 401706
Finished inferences for 401708
Finished inferences for 401714
Finished inferences for 401716
Finished inferences for 401718
Finished inferences for 401722
Finished inferences for 401724
Finished inferences for 401728
Finished inferences for 401734
Finished inferences for 401742
Finished inferences for 401748
Finished inferences for 401756
Finished

Finished inferences for 400182
Finished inferences for 400183
Finished inferences for 400184
Finished inferences for 400185
Finished inferences for 400186
Finished inferences for 400189
Finished inferences for 400190
Finished inferences for 400192
Finished inferences for 400193
Finished inferences for 400195
Finished inferences for 400201
Finished inferences for 400202
Finished inferences for 400203
Finished inferences for 400204
Finished inferences for 400206
Finished inferences for 400208
Finished inferences for 400209
Finished inferences for 400211
Finished inferences for 400212
Finished inferences for 400213
Finished inferences for 400214
Finished inferences for 400216
Finished inferences for 400218
Finished inferences for 400221
Finished inferences for 400222
Finished inferences for 400223
Finished inferences for 400224
Finished inferences for 400225
Finished inferences for 400227
Finished inferences for 400228
Finished inferences for 400231
Finished inferences for 400232
Finished

...

Finished inferences for 401689
Finished inferences for 401691
Finished inferences for 401692
Finished inferences for 401693
Finished inferences for 401694
Finished inferences for 401695
Finished inferences for 401696
Finished inferences for 401697
Finished inferences for 401698
Finished inferences for 401699
Finished inferences for 401700
Finished inferences for 401701
Finished inferences for 401702
Finished inferences for 401703
Finished inferences for 401704
Finished inferences for 401705
Finished inferences for 401706
Finished inferences for 401708
Finished inferences for 401714
Finished inferences for 401716
Finished inferences for 401718
Finished inferences for 401722
Finished inferences for 401724
Finished inferences for 401728
Finished inferences for 401734
Finished inferences for 401742
Finished inferences for 401748
Finished inferences for 401756
Finished inferences for 401770
Finished inferences for 401774
Finished inferences for 401778
Finished inferences for 401781
Finished

Finished inferences for 400186
Finished inferences for 400189
Finished inferences for 400190
Finished inferences for 400192
Finished inferences for 400193
Finished inferences for 400195
Finished inferences for 400201
Finished inferences for 400202
Finished inferences for 400203
Finished inferences for 400204
Finished inferences for 400206
Finished inferences for 400208
Finished inferences for 400209
Finished inferences for 400211
Finished inferences for 400212
Finished inferences for 400213
Finished inferences for 400214
Finished inferences for 400216
Finished inferences for 400218
Finished inferences for 400221
Finished inferences for 400222
Finished inferences for 400223
Finished inferences for 400224
Finished inferences for 400225
Finished inferences for 400227
Finished inferences for 400228
Finished inferences for 400231
Finished inferences for 400232
Finished inferences for 400235
Finished inferences for 400236
Finished inferences for 400237
Finished inferences for 400238
Finished

Finished inferences for 400696
Finished inferences for 400698
Finished inferences for 400699
Finished inferences for 400700
Finished inferences for 400703
Finished inferences for 400704
Finished inferences for 400707
Finished inferences for 400712
Finished inferences for 400713
Finished inferences for 400714
Finished inferences for 400715
Finished inferences for 400716
Finished inferences for 400717
Finished inferences for 400723
Finished inferences for 400726
Finished inferences for 400728
Finished inferences for 400731
Finished inferences for 400733
Finished inferences for 400734
Finished inferences for 400738
Finished inferences for 400739
Finished inferences for 400740
Finished inferences for 400741
Finished inferences for 400742
Finished inferences for 400743
Finished inferences for 400744
Finished inferences for 400745
Finished inferences for 400747
Finished inferences for 400748
Finished inferences for 400749
Finished inferences for 400750
Finished inferences for 400754
Finished

Finished inferences for 401357
Finished inferences for 401359
Finished inferences for 401362
Finished inferences for 401366
Finished inferences for 401371
Finished inferences for 401372
Finished inferences for 401373
Finished inferences for 401376
Finished inferences for 401378
Finished inferences for 401390
Finished inferences for 401391
Finished inferences for 401393
Finished inferences for 401394
Finished inferences for 401395
Finished inferences for 401399
Finished inferences for 401400
Finished inferences for 401401
Finished inferences for 401403
Finished inferences for 401405
Finished inferences for 401407
Finished inferences for 401408
Finished inferences for 401411
Finished inferences for 401412
Finished inferences for 401416
Finished inferences for 401421
Finished inferences for 401422
Finished inferences for 401424
Finished inferences for 401426
Finished inferences for 401431
Finished inferences for 401433
Finished inferences for 401434
Finished inferences for 401437
Finished

Finished inferences for 401883
Finished inferences for 401888
Finished inferences for 401889
Finished inferences for 401894
Finished inferences for 401898
Finished inferences for 401899
Finished inferences for 401900
Finished inferences for 401901
Finished inferences for 401908
Finished inferences for 401910
Finished inferences for 401911
Finished inferences for 401913
Finished inferences for 401914
Finished inferences for 401917
Finished inferences for 401918
Finished inferences for 401923
Finished inferences for 401924
Finished inferences for 401925
Finished inferences for 401926
Finished inferences for 401927
Finished inferences for 401928
Finished inferences for 401929
Finished inferences for 401934
Finished inferences for 401935
Finished inferences for 401936
Finished inferences for 401937
Finished inferences for 401942
Finished inferences for 401943
Finished inferences for 401950
Finished inferences for 401951
Finished inferences for 401954
Finished inferences for 401955
Finished